In [9]:
from dcicutils import ff_utils
from functions.notebook_functions import *
from functions.cleanup import *
import time
time1 = time.time()

#, store={}, item_uuids=[]
env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 



set_to_release = ['d864780f-006d-49d7-8de9-d7f3fea37608']


store, uuids = record_object_es(set_to_release, my_auth, schema_name, add_pc_wfr=True, store_frame='object')
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1))

90
90
4.221422910690308


In [10]:
# TODO
# Check audits

# do you want to check for duplicate/problematic runs on files?
# it will take some time
check_wfrs = True
delete_problematic = True

# check expsets
print('EXPSET CHECK')
for a_set in store['experiment_set_replicate']:
    if not a_set.get('completed_processes'):
        print(a_set['accession'], 'missing processing tag', a_set['description'][:50])

# check exps 
print('\nEXP CHECK')
# check for experiment numbers
exp_names = ['experiment_hi_c', 'experiment_mic', 'experiment_seq']
all_exps_on_sets = [a for i in store['experiment_set_replicate'] for a in i['experiments_in_set']]
all_exps = [a['uuid'] for i in store.keys() for a in store[i] if i in exp_names]
if len(all_exps_on_sets) != len(all_exps):
    print('Number of experiments is not same as experiments associated with sets')
    print('# of exps: {}. # of exps on sets: {}'.format(len(store['experiment_hi_c']), len(all_exps_on_sets)))
        
print('\nFILE FASTQ CHECK')
for a_file in store['file_fastq']:
    if not a_file.get('quality_metric'):
        print(a_file['accession'], 'missing fastqc')
    if not a_file.get('content_md5sum') or not a_file.get('md5sum'):
        print(a_file['accession'], 'problems with md5')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)

# check processed files
print('\nFILE PROCESSED CHECK')
for a_file in store['file_processed']:
    if a_file['file_format'] == '/file-formats/pairs/':
        if not a_file.get('quality_metric'):
            print(a_file['accession'], 'missing Pairsqc')
    if not a_file.get('source_experiments'):
        print(a_file['accession'], 'missing source experiments')
    if check_wfrs:
        dw = delete_wfrs(a_file, my_auth, delete=delete_problematic)   

# check wfrs
print('\nWFR CHECK')
for wfr in store['workflow_run_awsem']:
    if wfr['run_status'] != 'complete':
        print('problematic wfr', wfr['uuid'], wfr['run_status'])
        
# check for weird status
print('\nREPORT NUMBERS AND CHECK STATUS')
for i in store:
    print(i, len(store[i]))
    weird = [[i,x['uuid'],x['status']] for x in store[i] if x['status'] in ['deleted', 'replaced', 'archived', 'revoked']]
    if weird:
        for case in weird:
            print(case)
        print()


EXPSET CHECK
4DNESTCJSP7W missing processing tag Mapping RNA-genome interactions in H1 cell line us

EXP CHECK

FILE FASTQ CHECK

FILE PROCESSED CHECK

WFR CHECK

REPORT NUMBERS AND CHECK STATUS
file_fastq 6
protocol 2
user 11
award 4
file_processed 3
organism 1
workflow 4
file_format 5
experiment_seq 3
quality_metric_fastqc 6
individual_human 1
ontology 1
file_reference 1
image 1
lab 4
biosample_cell_culture 1
biosample 1
static_section 1
workflow_run_awsem 15
vendor 1
experiment_set_replicate 1
software 2
biosource 1
ontology_term 14


In [11]:
# Check status
change_status = 'released to project'

common_types = ['lab', 'user', 'award','ontology', 'ontology_term', 
                'file_format', 'software', 'workflow', 'static_section',
                'organism', 'vendor', 'file_reference', 'individual_human', 
                'enzyme', 'biosource', 'file_format']

print('COMMON ITEMS')
for a_type in store:
    if a_type in common_types:
        for raw_data in store[a_type]:
            if raw_data['status'] not in ['released', 'current']:
                print('COMMON ITEM NOT RELEASED', a_type,raw_data['status'], raw_data['uuid'])
print('---------------------------------\n')

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] != change_status:
            print(a_type, raw_data['uuid'], raw_data['status'])



COMMON ITEMS
COMMON ITEM NOT RELEASED static_section released to project 0c2ba23e-b256-47ce-a37c-0f1282471789
---------------------------------

file_fastq af3aaa42-3e35-4ecc-89ce-27d277744887 uploaded
file_fastq 82837fe1-9e76-44c9-ab0d-81ccff9bdb5e uploaded
file_fastq 7088a636-3095-439e-a86f-c9c8341ccd0a uploaded
file_fastq 45ac5982-3507-4e96-a7f6-4e2a70902bcf uploaded
file_fastq 341e265c-95ca-43ad-b627-742ab4ce65fe uploaded
file_fastq 2d2c9549-9288-4c8e-87f4-27d0e97e858a uploaded
protocol fa6738b1-ffc5-4fe6-b960-a1c3bbb26c2d released
protocol 50f8300d-400f-4ce1-8163-42f417cbbada released
file_processed adf4a4a7-c932-4636-a132-82bb6856a5f5 uploaded
file_processed ab59de07-50ba-4bf0-a7f4-310921d26cd4 uploaded
file_processed 4682e1b4-1b64-4eb8-bf5f-f15e881e659b uploaded
experiment_seq ffc3c6f2-e0ea-488e-a4a3-4bc219d3c4f6 in review by lab
experiment_seq d4d78643-bc72-4ef2-aac0-ae1fa111feaa in review by lab
experiment_seq 08f557c3-cb4e-4d66-8025-0c85b4d160f0 in review by lab
quality_metri

In [12]:
# Release the items
action = True
change_status = 'released to project'

counter = 0
for a_type in store:
    if a_type in common_types:
        continue
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print('deleted data', a_type ,raw_data['uuid'])
            continue        
        if raw_data['status'] not in ['current', 'released']:    
            counter +=1
            print(a_type, raw_data['status'], raw_data['uuid'])
            patch_data = {'status': change_status}
            if action:
                if change_status == 'released':
                    if a_type in ['publication', 'user', 'award', 'vendor', 'organism', ]:
                        ff_utils.patch_metadata({'status': 'current'}, obj_id=raw_data['uuid'] ,key=my_auth)
                    else:
                        ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                else:
                    ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,key=my_auth)
                
        else:
            print('ALREADY RELEASED/TO PROJECT', a_type, raw_data['status'], raw_data['uuid'])
            pass
            
print(counter)

file_fastq uploaded af3aaa42-3e35-4ecc-89ce-27d277744887
file_fastq uploaded 82837fe1-9e76-44c9-ab0d-81ccff9bdb5e
file_fastq uploaded 7088a636-3095-439e-a86f-c9c8341ccd0a
file_fastq uploaded 45ac5982-3507-4e96-a7f6-4e2a70902bcf
file_fastq uploaded 341e265c-95ca-43ad-b627-742ab4ce65fe
file_fastq uploaded 2d2c9549-9288-4c8e-87f4-27d0e97e858a
ALREADY RELEASED/TO PROJECT protocol released fa6738b1-ffc5-4fe6-b960-a1c3bbb26c2d
ALREADY RELEASED/TO PROJECT protocol released 50f8300d-400f-4ce1-8163-42f417cbbada
file_processed uploaded adf4a4a7-c932-4636-a132-82bb6856a5f5
file_processed uploaded ab59de07-50ba-4bf0-a7f4-310921d26cd4
file_processed uploaded 4682e1b4-1b64-4eb8-bf5f-f15e881e659b
experiment_seq in review by lab ffc3c6f2-e0ea-488e-a4a3-4bc219d3c4f6
experiment_seq in review by lab d4d78643-bc72-4ef2-aac0-ae1fa111feaa
experiment_seq in review by lab 08f557c3-cb4e-4d66-8025-0c85b4d160f0
quality_metric_fastqc in review by lab f32dbfa0-48d6-4c58-9c38-94adec3b1de9
quality_metric_fastqc in r